In [3]:
import pandas as pd
import numpy as np

__closepath__ = 'data/vn_stock/price_volume/close_matrix_20120101-20240101.txt'
__openpath__ = 'data/vn_stock/price_volume/open_matrix_20120101-20240101.txt'
__highpath__ = 'data/vn_stock/price_volume/high_matrix_20120101-20240101.txt'
__lowpath__ = 'data/vn_stock/price_volume/low_matrix_20120101-20240101.txt'
__volumepath = 'data/vn_stock/price_volume/volume_matrix_20120101-20240101.txt'

def load_and_process_data(file_path):
    df = pd.read_csv(file_path, sep='\t')
    df.set_index('time', inplace=True)
    df = df.astype(float)
    return df

close = load_and_process_data(__closepath__)
open = load_and_process_data(__openpath__)
high = load_and_process_data(__highpath__)
low = load_and_process_data(__lowpath__)
volume = load_and_process_data(__volumepath)

In [2]:
price = (high + low + close)/3
vwap = (price * volume).cumsum() / volume.cumsum()
vwap = vwap.round(2)
vwap.fillna(0, inplace=True)
vwap.to_csv('data/vn_stock/price_volume/vwap_matrix_20120101-20240101.txt', sep='\t')

In [3]:
adv20 = volume.rolling(window=20).mean()
adv20.fillna(0, inplace=True)
adv20.to_csv('data/vn_stock/price_volume/adv20_matrix_20120101-20240101.txt', sep='\t')

In [4]:
adv60 = volume.rolling(window=60).mean()
adv60.fillna(0, inplace=True)
adv60.to_csv('data/vn_stock/price_volume/adv60_matrix_20120101-20240101.txt', sep='\t')

In [5]:
adv120 = volume.rolling(window=120).mean()
adv120.fillna(0, inplace=True)
adv120.to_csv('data/vn_stock/price_volume/adv120_matrix_20120101-20240101.txt', sep='\t')

In [6]:
price = (high + low + close)/3
daily_return = price.pct_change()
daily_return.fillna(0, inplace=True)
daily_return.to_csv('data/vn_stock/price_volume/daily_return_matrix_20120101-20240101.txt', sep='\t')

In [2]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta, timezone

def get_vn30f1m_trading():
    start = '1325376000'
    end = '1704067200'
    url = f"https://services.entrade.com.vn/chart-api/chart?from={start}&resolution=1&symbol=VN30F1M&to={end}"
    try:
        resp = requests.get(url)
        data = resp.json()
        if not data:
            print(f"No data returned for VN30F1M")
        vn30f1m = pd.DataFrame(data).iloc[:, :6]
        vn30f1m['t'] = vn30f1m['t'].astype(int).apply(lambda x: (datetime.fromtimestamp(x, timezone.utc) + timedelta(hours=7)).replace(tzinfo=None))
        vn30f1m.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
        return vn30f1m
    except Exception as e:
        print(f"Error fetching data for VN30F1M: {e}.")
        return pd.DataFrame()
    
df = get_vn30f1m_trading()
df

,date,open,high,low,close,volume
0,2018-08-13 09:00:00,943.5,943.5,942.9,942.9,975
1,2018-08-13 09:01:00,943.0,943.1,942.9,943.1,220
2,2018-08-13 09:02:00,943.0,943.6,943.0,943.5,121
3,2018-08-13 09:03:00,943.3,943.4,943.3,943.4,135
4,2018-08-13 09:04:00,943.2,943.2,943.0,943.1,361
...,...,...,...,...,...,...
296481,2023-12-29 14:27:00,1133.2,1134.2,1133.0,1134.2,1232
296482,2023-12-29 14:28:00,1134.9,1134.9,1133.6,1133.6,1362
296483,2023-12-29 14:29:00,1133.9,1134.4,1133.5,1133.9,1458
296484,2023-12-29 14:30:00,1134.0,1134.0,1134.0,1134.0,76
